In [1]:
import pandas as pd
import numpy as np

import lib.transformers as tf
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier


In [2]:
cwruData0 = pd.read_parquet('data/cwru0.parquet')
cwruData1 = pd.read_parquet('data/cwru1.parquet')

cwruData = pd.concat([cwruData0, cwruData1])

In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

In [4]:
from skorch.callbacks import PassthroughScoring

In [5]:
torch.manual_seed(0)
device = torch.device("cuda:0") if torch.cuda.is_available() else 'cpu'

In [6]:
nz = 100  # size of the latent z vector
ngf = 32  # units of generator
ndf = 32  # units of discriminator
nc = 1  # number of channels
batch_size = 64
lr = 0.0002
beta1 = 0.5  # for adam
max_epochs = 5
ngpu = 1
isize = 32  # 32 is easier than 28 to work with
workers = 2  # for dataloader

In [7]:
from lib.model import Ganomaly, GanomalyNet

In [8]:
net = GanomalyNet(
    Ganomaly,
    module__isize = isize,
    module__nz=nz,
    module__ndf=ndf,
    module__ngf=ngf,
    module__nc=nc,
    module__ngpu=ngpu,
    
    criterion=nn.BCELoss,

    optimizer_gen=optim.Adam,
    optimizer_gen__lr=0.0002,
    optimizer_gen__betas=(beta1, 0.999),

    optimizer_dis=optim.Adam,
    optimizer_dis__lr=0.00002,
    optimizer_dis__betas=(beta1, 0.999),

    batch_size=batch_size,
    max_epochs=max_epochs,

    train_split=False,  # not implemented
    iterator_train__shuffle=True,
    iterator_train__num_workers=workers,
    iterator_valid__num_workers=workers,

    callbacks=[
        PassthroughScoring('loss_dis', on_train=True),
        PassthroughScoring('loss_gen', on_train=True),
        PassthroughScoring('loss_gen_fra', on_train=True),
        PassthroughScoring('loss_gen_app', on_train=True),
        PassthroughScoring('loss_gen_lat', on_train=True),
    ],
)

In [9]:
prep = \
Pipeline(steps=[('DataSelector', tf.DataSelector(columns = ['baseData', 'driveEndData'], condition = ['Normal Baseline'])),
                ('ArrayFlattener', tf.ArrayFlattener()),
                ('DatatypeFilter', tf.DataTypeFilter(data_type = np.ndarray)),
                ('ArrayChunker', tf.ArrayChunker(1024)),
                ('ArrayEqualizer', tf.ArrayEqualizer()),
                ('ArrayFlattener2', tf.ArrayFlattener()),
                ('RandomArraySampler', tf.RandomArraySampler(100, random_state = 0)),
                #('FeatureExtractor', tf.FeatureExtractor(axis=1)),
                ('ArrayReshaper', tf.ArrayReshaper((1, 32, 32))),
                ('GAN', net)
               ])

prep.fit(cwruData)

tensor([[[[ 0.0106,  0.0096,  0.0346,  ..., -0.1421, -0.0924, -0.0181],
          [ 0.0236,  0.0052, -0.0330,  ...,  0.0941,  0.0680,  0.0540],
          [ 0.0630,  0.0926,  0.1141,  ..., -0.0482, -0.0273, -0.0152],
          ...,
          [ 0.0390,  0.0515,  0.0547,  ...,  0.0159, -0.0355, -0.0555],
          [-0.0252,  0.0592,  0.1106,  ...,  0.0194,  0.0632,  0.0480],
          [-0.0081, -0.0774, -0.1039,  ...,  0.0567,  0.0386,  0.0378]]],


        [[[ 0.0449,  0.1185,  0.1281,  ...,  0.0730,  0.0371, -0.0073],
          [-0.0259,  0.0060,  0.0565,  ..., -0.0113,  0.0284,  0.0334],
          [ 0.0090, -0.0446, -0.0862,  ...,  0.0544,  0.0373,  0.0236],
          ...,
          [-0.0194, -0.0242, -0.0046,  ..., -0.0344,  0.0131,  0.0857],
          [ 0.1056,  0.0784,  0.0461,  ..., -0.0319, -0.0891, -0.1195],
          [-0.0843, -0.0207,  0.0259,  ...,  0.0599,  0.0515, -0.0146]]],


        [[[ 0.0615,  0.0255, -0.0038,  ...,  0.0818,  0.0411, -0.0027],
          [-0.0094,  0.010

TypeError: Device(): argument 'type' (position 1) must be str, not NoneType